<a href="https://colab.research.google.com/github/kunalbathija/Udemy-TF-2.0/blob/master/Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [3]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

170500096/170498071 [==============================] - 6s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [4]:
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [0]:
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [6]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [7]:
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [8]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [12]:
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15)

Epoch 1/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.4265 - accuracy: 0.8533 - val_loss: 0.6553 - val_accuracy: 0.7838
Epoch 2/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.3575 - accuracy: 0.8758 - val_loss: 0.6445 - val_accuracy: 0.7950
Epoch 3/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.2991 - accuracy: 0.8968 - val_loss: 0.6292 - val_accuracy: 0.8137
Epoch 4/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.2632 - accuracy: 0.9098 - val_loss: 0.6260 - val_accuracy: 0.8199
Epoch 5/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.2202 - accuracy: 0.9231 - val_loss: 0.7620 - val_accuracy: 0.7966
Epoch 6/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.2027 - accuracy: 0.9305 - val_loss: 0.6899 - val_accuracy: 0.8163
Epoch 7/15
1563/1563 [==============================] - 15s 10ms/step - loss: 0.1709 - accuracy: 0.9404 - val_loss: 0.7205 -

In [10]:
50000/391

127.8772378516624

In [11]:
10000/79

126.58227848101266

In [13]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2995 - accuracy: 0.5534 - val_loss: 1.0762 - val_accuracy: 0.6356
Epoch 2/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8431 - accuracy: 0.7051 - val_loss: 0.9222 - val_accuracy: 0.6885
Epoch 3/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.6918 - accuracy: 0.7623 - val_loss: 0.7121 - val_accuracy: 0.7545
Epoch 4/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5900 - accuracy: 0.7980 - val_loss: 0.6989 - val_accuracy: 0.7691
Epoch 5/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.4928 - accuracy: 0.8298 - val_loss: 0.6311 - val_accuracy: 0.7972
